# Decode `.fit` data
Play around and try to decode `.fit` data from Garmin devices. Following instructions taken from [here](https://www.deuxpi.ca/blog/activity-tracking-files.html)

In [1]:
import fitparse
import sys

In [6]:
input_file = '/Users/francesco/PersonalProjects/data/garmin/2019-05-01/33802246536.fit'
fitfile = fitparse.FitFile(input_file, data_processor=fitparse.StandardUnitsDataProcessor())

In [7]:
type(fit_file)

fitparse.base.FitFile

In [23]:
data_type = []
for record in fitfile.get_messages():
    
    info = record.name
    if record.type == 'data':
        info = info + ' this is data!'
        data_type.append(record.type)
    
    #print(info)
    #if record.type == 'data':
    #    print(record.name)
    #    for field_data in record:
    #        print(' * {}: {}'.format(field_data.name, field_data.value))

file_id
 * garmin_product: 2623
 * manufacturer: garmin
 * number: 135
 * serial_number: 3964402232
 * time_created: 2019-04-30 22:00:00
 * type: monitoring_b
 * unknown_6: None
device_info
 * garmin_product: 2623
 * manufacturer: garmin
 * serial_number: 3964402232
 * software_version: 3.9
 * timestamp: 2019-04-30 22:00:00
software
 * version: 4.5
monitoring_info
 * activity_type: (6, 1)
 * cycles_to_calories: (0.0542, 0.171)
 * cycles_to_distance: (1.6652, 2.4978)
 * local_timestamp: 2019-05-01 00:00:00
 * resting_metabolic_rate: 2166
 * timestamp: 2019-04-30 22:00:00
 * unknown_7: (10400, 10400)
 * unknown_8: 1
monitoring
 * timestamp: 2019-04-30 22:00:00
 * unknown_35: 0
 * unknown_36: 0
monitoring
 * timestamp: 2019-04-30 22:00:00
 * unknown_37: 0
 * unknown_38: 0
ohr_settings
 * enabled: on
 * unknown_253: 925596000
unknown_233
 * unknown_2: (0, 0, 0, 40)
monitoring
 * activity_type: sedentary
 * current_activity_type_intensity: (8,)
 * intensity: 0
 * timestamp: 2019-04-30 22:00

In [31]:
data_type_unique = list( set(data_type) )
print('records can be of {} type(s):'.format(len(data_type_unique)), data_type_unique)

records can be of 1 type(s): ['data']
